In [ ]:
import anndata as ad
import os
import pandas as pd
import scanpy as sc
import subprocess
from scipy import sparse
import numpy as np

portal_obs_fields = [
    'assay',
    'cell_type',
    'development_stage',
    'disease',
    'self_reported_ethnicity',
    'organism',
    'sex',
    'tissue'
]
full_obs_standards = portal_obs_fields + [e + '_ontology_term_id' for e in portal_obs_fields] + ['donor_id','suspension_type','is_primary_data']

In [ ]:
full_obs_standards

We need to load the following libraries. Make sure that the anndata version you have installed matches the one used by [cellxgene-schema](https://github.com/chanzuckerberg/single-cell-curation)

In [ ]:
pip show anndata

In [ ]:
pip show cellxgene-schema 


# Loading the AnnData object
**Update the name of the file (without the .h5ad extension)**<br>
*The sample `my_matrix.h5ad` that is in this repo is subsampled from https://cellxgene.cziscience.com/e/f15e263b-6544-46cb-a46e-e33ab7ce8347.cxg/ with some metadata alterations for the purpose of this tutorial*

In [ ]:
file = 'lung_5loc_sc_sn_cellxgene_23092022'

**Load the AnnData object**

In [ ]:
adata = sc.read_h5ad(file + '.h5ad')
adata.raw.X 

# data layers
**If needed, transfer to sparse matrix format, this saves space**

count_nonzero = counts the number of non-zero values in the array 

In [ ]:
def determine_sparsity(x):
    if isinstance(x, sparse.coo_matrix) or isinstance(x, sparse.csr_matrix) or isinstance(x, sparse.csc.csc_matrix):
        sparsity = 1 - x.count_nonzero() / float(np.cumprod(x.shape)[-1])
    elif isinstance(x, np.ndarray):
        sparsity = 1 - np.count_nonzero(x) / float(np.cumprod(x.shape)[-1])
    else:
        print(f'matrix is of type {type(x)}, sparsity calculation has not been implemented')

    return sparsity


max_sparsity = 0.5

sparsity = determine_sparsity(adata.X)
print(f'X sparsity: {sparsity}')
if sparsity > max_sparsity and type(adata.X) != sparse.csr.csr_matrix:
    print('converting X to sparse')
    adata.X = sparse.csr_matrix(adata.X)

if adata.raw:
    sparsity = determine_sparsity(adata.raw.X)
    print(f'raw.X sparsity: {sparsity}')
    if sparsity > max_sparsity and type(adata.raw.X) != sparse.csr.csr_matrix:
        print('converting raw.X to sparse')
        raw_adata = ad.AnnData(adata.raw.X, var=adata.raw.var, obs=adata.obs)
        raw_adata.X = sparse.csr_matrix(raw_adata.X)
        adata.raw = raw_adata
        del raw_adata

for l in adata.layers:
    sparsity = determine_sparsity(adata.layers[l])
    print(f'layers[{l}] sparsity: {sparsity}')
    if sparsity > max_sparsity and type(adata.layers[l]) != sparse.csr.csr_matrix:
        print(f'converting layers[{l}] to sparse')
        adata.layers[l] = sparse.csr_matrix(adata.layers[l])

**Check the min/max of each layer**<br>
*Look for duplicated or other unnecessary layers*<br>
*Raw should be whole, positive, ~10<sup>3*

In [ ]:
if adata.raw:
    print('raw min = ' + str(adata.raw.X.min()))
    print('raw max = ' + str(adata.raw.X.max()))
    non_integer = np.any(~np.equal(np.mod(adata.raw.X.data, 1), 0))
else:
    non_integer = np.any(~np.equal(np.mod(adata.X.data, 1), 0))

if non_integer == False:
    print('raw is all integers')
else:
    print('ERROR: raw contains non-integer values')

print('X min = ' + str(adata.X.min()))
print('X max = ' + str(adata.X.max()))

for l in adata.layers:
    print(f'layers[{l}] min = ' + str(adata.layers[l].min()))
    print(f'layers[{l}] max = ' + str(adata.layers[l].max()))

**Sample:** delete a layer

In [ ]:
del adata.layers['other_raw']

# obsm
**Confirm at least one set of embeddings is present**

In [ ]:
adata.obsm

**View embeddings to identify which matches paper figures**

In [ ]:
sc.set_figure_params(dpi=100)
for e in adata.obsm:
    sc.pl.embedding(adata, basis=e, color='Celltypes_colors', legend_loc='on data')

**Check that the default_embedding value, if defined, is in obsm**

In [ ]:
if 'default_embedding' in adata.uns:
    de = adata.uns['default_embedding']
    if de not in adata.obsm_keys():
        print('ERROR:' + de + ' not in ' + ','.join(adata.obsm_keys()))
    else:
        print(de + ' is in ' + ','.join(adata.obsm_keys()))

**Sample:** update uns.default_embedding\
Ideally, the default_embedding matches the figures in the paper

In [ ]:
adata.uns['default_embedding'] = 'X_umap'

In [ ]:
adata.uns

# uns
**Check for uns schema fields**

In [ ]:
uns_schema =['schema_version','title']
for p in uns_schema:
    print(p + ': ' + adata.uns.get(p,'MISSING'))

**Sample:** define uns.schema_version

In [ ]:
adata.uns['schema_version'] = '3.0.0'

In [ ]:
adata.uns['title'] = 'hello'

**Browse all of uns**

In [ ]:
adata.uns

**Sample:** remove deprecated schema field

In [ ]:
del adata.uns['X_normalization']

# *_colors
**scanpy & cellxgene allow for specification of cluster colors when coloring by specific obs fields**<br>
**A list of color codes is specified in `uns.PROP_colors` where `PROP` is an obs field**<br>
**The number of color codes in `uns.PROP_colors` must be at least as long as the number of unique values in `obs.PROP`**<br>
<br>
**Check for _colors fields & ensure each matches a categorical obs field**

In [ ]:
numb_types = ['int_', 'int8', 'int16', 'int32', 'int64', 'uint8', 'uint16', 'uint32', 'uint64','float_', 'float16', 'float32', 'float64']

for k in adata.uns.keys():
    if k.endswith('_colors'):
        colors = len(adata.uns[k])
        obs_field = k[:-(len('_colors'))]

        if obs_field.endswith('_ontology_term_id'):
            label_field = obs_field[:-17]
            print(f'WARNING: consider copying uns.{k} to uns.{label_field}_colors so palette transfers to CxG viz')

        if obs_field in portal_obs_fields:
            obs_field += '_ontology_term_id'
        if obs_field not in adata.obs.keys():
            print(f'WARNING: {obs_field} not found in obs, consider DELETING or RENAMING uns.{k}')
        else:
            values = len(adata.obs[obs_field].unique())
            if colors < values:
                print(f'ERROR: uns.{k} has only {str(colors)} colors but obs.{obs_field} has {str(values)} values')
            if adata.obs.dtypes[obs_field].name in numb_types:
                print(f'ERROR: uns.{k} is associated with non-categorical {obs_field}')

# obs

In [ ]:
adata.obs_keys()

In [ ]:
adata.obs

**Ensure the portal fields are not used**<br>
**Ensure schema fields are present and values are valid & precise**

In [ ]:
for o in full_obs_standards:
    print(o)
    if o not in adata.obs_keys():
        print('NOT IN OBS')
    else:
        un = adata.obs[o].unique()
        if un.dtype == 'category':
            print(un.to_list())
        else:
            print(un.tolist())

### Prepare the obs layer

First we extract the obs layer from the h5ad file that has been provided to us. Then we extract out the obs layer as a csv file. 



In [ ]:
a_obs = adata.obs
a_obs.to_csv("a_obs_layer.csv")


In [ ]:
r_obs = adata.raw.obs
r_obs.to_csv("r_obs_layer.csv")

Download the excel spreadsheet from the INGEST data submission, and navigate to the sequence_file page.
Select the cell_suspension.biomaterial_core.biomaterial_id, cell_suspension.uuid, and library_preparation_protocol.protocol_core.protocol_id and filter for duplicates on those keys. This results in a unique set of cell_suspensions with their related data. 

See script here: https://github.com/ebi-ait/ingest-cellxgene-submitter#create-obs-layer-from-multiple-cell-suspension-uuids

The manual work here is  matching up unique HCA cell_suspensions and metadata with the  unique samples in the provided-h5ad file. We then generate a final obs_layer.csv which combines HCA metadata with the provided metadata from the contributor in matching rows. There is an opportunity for future automation here for scripts to perform the matching and to provide cell-type ontology terms. 

For a list of fields which should be in the final obs_layer.csv see (https://github.com/chanzuckerberg/single-cell-curation/blob/main/schema/2.0.0/schema.md#obs-cell-metadata)
- DCP_Cell_suspension_uuid
- DCP_Cell_suspension_ID
- assay_ontology_term_id
- cell_type_ontology_term_id
- development_stage_ontology_term_id
- disease_ontology_term_id
- self_reported_ethnicity_ontology_term_id
- is_primary_data
- organism_ontology_term_id
- sex_ontology_term_id
- tissue_ontology_term_id
- donor_id

There is also some manual work here to select cell-type ontology terms from the free text cell-type provided by the contributors. There is an opportunity for future automation here to get scripts / work with other experts in ontology to do this. 

Once we have the final combined obs_layer, then save it as "obs_layer.csv" and save it as a dobs object

In [ ]:
dobs = pd.read_csv("a_obs_layer.csv",sep=",")

In [ ]:
for o in full_obs_standards:
    print(o)
    if o not in dobs.keys():
        print('NOT IN OBS')
    else:
        un = dobs[o].unique()
        if un.dtype == 'category':
            print(un.to_list())
        else:
            print(un.tolist())

In [ ]:
adata.obs = dobs

Check here for redundant fields in the obs layer (e.g. tissue_label, donor_ext). Assay_ontology_term_id should be '10x v2 5' sequencing' as opposed to '10x TCR'.
Development_stage and tissue should be made specific (e.g. 35-year old human stage).
Unannotated cells use CL:0000003 (native cell).
Check that certain fields in the obs layer should not be continuous / numerical, but should be categorical or string. Use the cells below to fix this if necessary.

**Sample:** set the index to the "barcodes" column

In [ ]:
dobs = dobs.set_index("Unnamed: 0", inplace = False)

**Sample:** drop "author_tissue" and "donor_ext" columns from obs

In [ ]:
adata.obs = adata.obs.drop(['Unnamed: 18','author_tissue','tissue_label','donor_ext'], axis = 1)

**Sample:** set specific obs datatypes to category

In [ ]:
adata.obs = adata.obs.astype({'batch': 'category','cluster':'category'}, copy = False)

**Ensure the portal fields are not used**<br>
**Ensure schema fields are present and values are valid & precise**

In [ ]:
for o in full_obs_standards:
    print(o)
    if o not in adata.obs_keys():
        print('NOT IN OBS')
    else:
        un = adata.obs[o].unique()
        if un.dtype == 'category':
            print(un.to_list())
        else:
            print(un.tolist())

# 10x barcode checker
**Checks a random 1k barcodes against 10x barcode lists**<br>
*Can help confirm 3' v2 vs v3 vs multiome*<br>
*5' v1 and v2 kits use the same barcode list as 3' v2*<br>
*Assumes the barcode is in the index. Suffixes/prefixes are OK*<br>
<br>
**Define the function**

In [ ]:
import json
import re
from random import randint


def TENx_barcode_checker(df):
    num_to_check = 1000

    v2_file = '/Users/wteh/Documents/Wrangling/Cellxgene/Notebooks/737K-august-2016.txt'
    v3_file = '/Users/wteh/Documents/Wrangling/Cellxgene/Notebooks/3M-february-2018.txt'
    multiome_file = '/Users/wteh/Documents/Wrangling/Cellxgene/Notebooks/737K-arc-v1.txt'

    v2_list = [line.strip() for line in open(v2_file, 'r')]
    v3_list = [line.strip() for line in open(v3_file, 'r')]
    multiome_list = [line.strip() for line in open(multiome_file, 'r')]

    cellcount = df.index.shape[0]
    barcode_pattern = '[ACTG]{16}'
    barcode_results = ''
    if re.search(barcode_pattern, df.index[5]):
        cellcount
        random_indices = [randint(0, cellcount - 1) for p in range(0, num_to_check)]
        barcodes = {'3pv2_5pv1_5pv2': 0,'3pv3': 0,'multiome': 0,'multiple': 0,'none': 0}
        for i in random_indices:
            if re.search(barcode_pattern, df.index[i]):
                barcode = re.search(barcode_pattern, df.index[i]).group(0)
                if barcode in v2_list:
                    if barcode in multiome_list:
                        barcodes['multiple'] += 1
                    elif barcode in v3_list:
                        barcodes['multiple'] += 1
                    else:
                        barcodes['3pv2_5pv1_5pv2'] += 1
                elif barcode in multiome_list:
                    if barcode in v3_list:
                        barcodes['multiple'] += 1
                    else:
                        barcodes['multiome'] += 1
                elif barcode in v3_list:
                    barcodes['3pv3'] += 1
                else:
                    barcodes['none'] += 1
        return barcodes

**Check 1k barcodes across the whole obs**

In [ ]:
results = TENx_barcode_checker(adata.obs)
if not results:
    print('no barcodes checked')
pd.DataFrame([results])

**Additionally, can check 1k barcodes from multiple subsets of obs**<br>
*Define `prop` and 1k barcodes will be checked for each unique value in `obs.prop`*

In [ ]:
prop = 'assay_ontology_term_id'

results = []
for a in adata.obs[prop].value_counts().keys():
    print(a)
    r = TENx_barcode_checker(adata.obs[adata.obs[prop] == a])
    if r:
        r[prop] = a
        results.append(r)
    else:
        print('no barcodes checked')
pd.DataFrame(results).set_index(prop)

**Sample:** set a column with all the same values

In [ ]:
adata.obs['is_primary_data'] = True
adata.obs['suspension_type'] = 'nucleus'

**Sample:** change column names

In [ ]:
rename_me = {
    'cell_type': 'author_cell_type',
    'ethnicity_ontology_id': 'self_reported_ethnicity_ontology_term_id',
    'disease_ontology_id': 'disease_ontology_term_id'
}

adata.obs.rename(columns=rename_me, inplace=True)

**Sample:** fill null values of a specific column with a specified value

In [ ]:
adata.obs['sex_ontology_term_id'].cat.add_categories('unknown', inplace=True)
adata.obs.fillna({'sex_ontology_term_id': 'unknown'}, inplace=True)
adata.obs['sex_ontology_term_id'].value_counts(dropna=False)

**Sample:** adjust the values in a specific column in a standard way

In [ ]:
def fix_typo(x):
    return x.replace('_',':')


adata.obs['development_stage_ontology_term_id'] = adata.obs['development_stage_ontology_term_id'].apply(fix_typo)
adata.obs['development_stage_ontology_term_id'].value_counts(dropna=False)

**Sample:** replace specified values in specified columns

In [ ]:
replace_me = {
    'organism_ontology_term_id':{'human':'NCBITaxon:9606', 'mouse': 'NCBITaxon:10090'},
    'assay_ontology_term_id': {'EFO:0030003': 'EFO:0009899'}
}

adata.obs.replace(replace_me,inplace=True)
adata.obs[['organism_ontology_term_id','assay_ontology_term_id']].value_counts(dropna=False)

**Sample:** add a new column with values based on values in an existing column - with DataFrame<br>
**Step 1:** get the values to map from

In [ ]:
for k in adata.obs['author_cell_type'].unique():
    print(k)

**Sample:** add a new column with values based on values in an existing column<br>
**Step 2:** set up a dataframe with the mapping
**Option A:** from a dictionary

In [ ]:
#map in values based on another field - step 2: set up a dataframe with the mapping
#option A: from dict
celltypes = {
    'Myeloid': 'CL:0001082',
    'Endothelial': 'CL:0010008',
    'Fibroblast': 'CL:0002548',
    'Cardiomyocyte': 'CL:0000513',
    'Pericyte': 'CL:0000669',
    'Lymphoid': 'CL:0000838',
    'Cycling cells': 'CL:0000003',
    'vSMCs': 'CL:0000514',
    'Neuronal': 'CL:0000006'
}

ct_df = pd.DataFrame.from_dict(celltypes,orient='index',columns=['cell_type_ontology_term_id']).reset_index().rename(columns={'index':'author_cell_type'})
ct_df

**Sample:** add a new column with values based on values in an existing column<br>
**Step 2:** set up a dataframe with the mapping<br>
**Option B:** from a Google Sheet<br>
*Google Sheet permissions must be Anyone with Link is a Viewer*

In [ ]:
sheet_id = '15oG8v5BS6HMPqCehYQcujMZUq9PgQNpo8osKhO7yA5o'
tab_name = 'Sheet1'
url = f'https://docs.google.com/spreadsheets/d/{sheet_id}/gviz/tq?tqx=out:csv&sheet={tab_name}'
ct_df = pd.read_csv(url)
ct_df

**Sample:** add a new column with values based on values in an existing column<br>
**Step 3:** merge the dataframe into obs<br>
*`how='left'` is critical to ensure obs order is retained*<br>
*`set_index` is critical to ensure the index is retained*<br>

In [ ]:
adata.obs = adata.obs.merge(ct_df, on='author_cell_type',how='left').set_index(adata.obs.index)
adata.obs[['author_cell_type','cell_type_ontology_term_id']].value_counts(dropna=False)

**Sample:** add a new column with values based on values in an existing column - with Dictionary<br>

In [ ]:
adata.obs['sample'].value_counts()

In [ ]:
donor_map = {
    'KL001': 'P21',
    'KL002': 'P22',
    'KL003': 'P23'
}

adata.obs['donor_id'] = adata.obs['sample'].map(donor_map)
adata.obs[['donor_id','sample']].value_counts(dropna=False)

**Look for general obs field issues and collect obs information to check for redundant information**

In [ ]:
long_fields = []
gradient_fields = []
uber_dict = {}
for o in adata.obs.keys():
    vc_dict = adata.obs[o].value_counts(dropna=False).to_dict()
    counts = '_'.join([str(c) for c in vc_dict.values()])
    count_len = len(vc_dict.keys())
    values = [str(i) for i in vc_dict.keys()]

    if o.startswith(' ') or o.endswith(' ') or '  ' in o:
        print('leading/trailing whitespace:' + o)

    if o not in full_obs_standards and ' '.join(o.split()).lower() in full_obs_standards:
        print('schema conflict:' + o)

    if count_len == 1:
        lone_v = str(list(vc_dict.keys())[0])
        if o not in full_obs_standards:
            print('all same value:' + o + ',' + lone_v)

    numb_types = ['int_', 'int8', 'int16', 'int32', 'int64', 'uint8', 'uint16', 'uint32', 'uint64','float_', 'float16', 'float32', 'float64']
    if adata.obs.dtypes[o].name in numb_types:
        gradient_fields.append(o)
    #check for long categories as they will not be enabled for coloring
    elif count_len > 200:
        long_fields.append(o)

    #report value_counts to later look for redundancy
    metadata = {
        'values': values,
        'property': o
    }
    if counts in uber_dict:
        uber_dict[counts].append(metadata)
    else:
        uber_dict[counts] = [metadata]

**Comb value_counts to report possible redundancy**

In [ ]:
for k,v in uber_dict.items():
    if '_' in k and not k.startswith('1_1'):
        props = [e['property'] for e in v]
        if len(v) > 1 and not all(elem in full_obs_standards for elem in props):
            print('cells breakdown: ' + k)
            for e in v:
                print(e['property'])
                #print(e['values'])
            print('----------------------------------------------------------------------------')

**Investigate any fields that may be redundant**

In [ ]:
adata.obs[['sample','patient','age','development_stage_ontology_term_id']].value_counts(dropna=False)

**Check for fields that aren't appropriate as gradient (e.g. cluster number)**

In [ ]:
gradient_fields

**Update a gradient field to categorical, if needed**

In [ ]:
adata.obs['cluster_id'] = adata.obs['cluster_id'].map(str)

**List any categorical fields with more than 200 categories as they may not be useful in the visualization**

In [ ]:
long_fields

**List any obs columns to remove and remove them**

In [ ]:
obs_remove = [
    'tissue',
    'organism',
    'self_reported_ethnicity',
    'assay',
    'disease',
    'sex',
    'cell_type'
]

obs_remove = [o for o in obs_remove if o in adata.obs.columns]
adata.obs.drop(columns=obs_remove, inplace=True)
if obs_remove:
    print('removed: ' + ','.join(obs_remove))

**Ensure the portal fields are not used**<br>
**Ensure schema fields are present and values are valid & precise**

In [ ]:
for o in full_obs_standards:
    print(o)
    if o not in adata.obs_keys():
        print('NOT IN OBS')
    else:
        un = adata.obs[o].unique()
        if un.dtype == 'category':
            print(un.to_list())
        else:
            print(un.tolist())

In [ ]:
adata.obs

# var
**Check for Ensembl IDs, redundant fields, etc.**<br>
**Check for schema fields**

## Map Gene Symbols to ENSEMBL IDs
**Skip this section if there are already ENSEMBL IDs**

In [ ]:
adata.var

In [ ]:
adata.raw.var

In [ ]:
raw_adata = ad.AnnData(adata.raw.X, var=adata.raw.var, obs=adata.obs)


**Set ENSEMBL IDs as barcodes**

In [ ]:
adata.var = adata.var.set_index("gene_ids-1", inplace = False)

In [ ]:
raw_adata.var = raw_adata.var.set_index("gene_ids-1", inplace = False)

**If CellRanger may have been used for alignment, check against the default CellRanger references for matches in order to inform symbol-to-ID mapping**

In [ ]:
CR_12 = '/Users/wteh/Documents/Wrangling/Cellxgene/Notebooks/refdata-cellranger-GRCh38-1_2_0_genes_gtf.tsv'
CR_30 = '/Users/wteh/Documents/Wrangling/Cellxgene/Notebooks/refdata-cellranger-GRCh38-3_0_0_genes_gtf.tsv'
CR_2020 = '/Users/wteh/Documents/Wrangling/Cellxgene/Notebooks/refdata-gex-GRCh38-2020-A_genes_gtf.tsv'
CR_hg19 = '/Users/wteh/Documents/Wrangling/Cellxgene/Notebooks/refdata-cellranger-hg19-1_2_0_genes_gtf.tsv'
for v in [CR_12,CR_30,CR_2020,CR_hg19]:
    map_df = pd.read_csv(v, sep='\t')
    print(v)
    print(adata.var.merge(map_df,left_index=True,right_on='gene_symbols',how='inner').shape[0])
    print('----------')

**Fill in the mapping file to use to map symbols to Ensembl IDs**<br>
*Expecting a .tsv with columns `gene_symbols` & `gene_ids`*

In [ ]:
var_mapping_file = CR_12

**View what features are not mapped in this**<br>
*Check for typos or other alterations to the symbols that can be fixed*<br>
*Common to see many ending in `.1` or `-1` resulting from duplicated symbols in the reference*

Shows list of gene features that are not properly mapped 

In [ ]:
var_map_df = pd.read_csv(var_mapping_file, sep='\t')
adata.var[adata.var.index.isin(var_map_df['gene_symbols']) != True]

### Add Filtered out genes from raw to normalized

**Skip this section if there is the identical number of genes in raw and normalized (if cell below = True)**

- If any genes genes have been filtered out when the authors processed the raw matrix, they will not available in the annotated matrix. We need to add them to the processed matrix. We checked this by inspecting the matrices, but can be checked again by running the following cell:
- Check that both dataframes have the same number of rows. If they are different, the authors filtered out some genes from the PROCESSED, and we will need to add them in. 


In [ ]:
dvar = pd.DataFrame(data=adata.var)
rvar = pd.DataFrame(data=raw_adata.var)
dvar.shape[0] == rvar.shape[0]

If some of the features have been filtered out of the processed matrices, we have to add the filtered-out genes at the end of the matrices. For that, we are going to first fill in the *feature_is_filtered* column at the rvar dataframe. We can then create a new dataframe dropping all the non filtered gene, and add this dataframe with the filtered genes at the end of dvar

In [ ]:
genes_add = [x for x in rvar.index.to_list() if x not in adata.var.index.to_list()]
all_genes = adata.var.index.to_list()
all_genes.extend(genes_add)

In [ ]:
genes_add

In [ ]:
all_genes

In [ ]:
new_var = pd.DataFrame(index=all_genes)
new_var = pd.merge(new_var, rvar, left_index=True, right_index=True, how='left')
new_var['feature_is_filtered'] = False
new_var.loc[genes_add, 'feature_is_filtered'] = True

In [ ]:
new_var

**Also editing the X layer to add filtered out genes from raw to normalized matrix**

In [ ]:
dExprs = pd.DataFrame(data=adata.X)
dExprs = dExprs.set_axis(dvar.index.to_list(), axis=1, inplace=False)


In [ ]:
dExprs

In [ ]:
dExprsgenesToAdd = new_var.loc[new_var['feature_is_filtered'] == True]
dExprs = dExprs.reindex(columns=[*dExprs.columns.tolist(), *dExprsgenesToAdd.index.to_list()], fill_value=0.0)

In [ ]:
dExprs

**Similar checks for raw.var, if present**

In [ ]:
rExprs = pd.DataFrame(data=raw_adata.X.toarray())


In [ ]:
rExprs = rExprs.set_axis(rvar.index.to_list(), axis = 1, inplace=False)
rExprs = rExprs.reindex(columns = dExprs.columns)


In [ ]:
rvar = rvar.reindex(index=dExprs.columns)


In [ ]:
rvar

In [ ]:
rExprs

In [ ]:
dExprs

## Filter out not-approved genes

**Create the list of approved IDs to filter on**<br>
*For the initial run, download the 4 genes_ csv files from https://github.com/chanzuckerberg/single-cell-curation/tree/main/cellxgene_schema_cli/cellxgene_schema/ontology_files*<br>
*After that, if the `genes_approved.csv` is available locally, then the 4 genes_ files won't be necessary*

In [ ]:
ref_files = [
    'genes_ercc.csv',
    'genes_homo_sapiens.csv',
    'genes_mus_musculus.csv',
    'genes_sars_cov_2.csv'
]

if not os.path.exists('genes_approved.csv'):
    ids = pd.DataFrame()
    for f in ref_files:
        df = pd.read_csv(f, names=['feature_id','symb','num','length'],dtype='str',index_col=False)
        ids = ids.append(df)
        os.remove(f)
    ids.to_csv('genes_approved.csv', index=False)

approved = pd.read_csv('genes_approved.csv',dtype='str')

**Sample:** set columns with all the same values

In [ ]:
adata.var['feature_is_filtered'] = False

**Remove any fields (typically symbols as the portal will add those)**

In [ ]:
var_remove = [
    'gene_symbols'
]

adata.var.drop(columns=var_remove, inplace=True)

**Map the Ensembl IDs**

In [ ]:
adata.var = adata.var.merge(var_map_df,left_index=True,right_on='gene_symbols',how='left').set_index(adata.var.index)

**Filter out genes that don't appear in the approved annotation**

In [ ]:
var_to_keep = adata.var.index.tolist()
var_in_approved = adata.var.index[adata.var['gene_ids'].isin(approved['feature_id'])].tolist()
var_to_keep = [e for e in var_to_keep if e in var_in_approved]
adata = adata[:, var_to_keep]
adata.var.set_index('gene_ids',inplace=True)

In [ ]:
adata.var

**Repeat much of the same steps for the `raw.var`, if it exists**

In [ ]:
rvar

In [ ]:
rExprs.columns

In [ ]:
rvar.index

In [ ]:
raw_var_remove = [
    'gene_ids-0'
]
rvar.drop(columns=raw_var_remove, inplace=True)

raw_adata = ad.AnnData(rExprs, var=rvar, obs=adata.obs)



In [ ]:
raw_adata.var = raw_adata.var.merge(var_map_df,left_index=True,right_on='gene_symbols',how='left').set_index(raw_adata.var.index)

raw_adata = raw_adata[:, var_to_keep]
raw_adata.var.set_index('gene_ids',inplace=True)
adata.raw = raw_adata
adata.raw.var

# Validate
**Determine the embedding by which to plot**\
May need to overwrite if the first obsm is not informative

In [ ]:
default_embedding = adata.uns.get('default_embedding')
umap_embedding = None
tsne_embdding = None
for k in adata.obsm_keys():
    if 'umap' in k.lower():
        umap_embedding = k
    elif 'tsne' in k.lower():
        tsne_embdding = k
if not default_embedding:
    if umap_embedding:
        default_embedding = umap_embedding
    elif tsne_embdding:
        default_embedding = tsne_embdding
    else:
        default_embedding = adata.obsm_keys()[0]
default_embedding

**Plot the cells to ensure they cluster by cell type**

In [ ]:
sc.set_figure_params(dpi=150)
sc.pl.embedding(adata, basis=default_embedding, color=['cell_type_ontology_term_id'])

**The above plot will set a color palette in uns, so remove that**

In [ ]:
del adata.uns['cell_type_ontology_term_id_colors']

**Plot by multiple genes using the normalized counts**<br>
*It is best to get a list of genes relevant to the specific data from the contributor/publication*

**Compare with the same genes using the raw counts to confirm they are correlated**

**Additionally, you could compare dotplots of those genes in each cell population**<br>
*This will scale all genes based on the max range of any gene so 1 gene with high values may make others difficult to distinguish*

**Write the file**

In [ ]:
new_one = file + '_revised.h5ad'
adata.write(filename=new_one, compression='gzip')
new_one

**Run the CELLxGENE validator**

In [ ]:
validate_process = subprocess.run(['cellxgene-schema', 'validate', new_one], stdout=subprocess.PIPE, stderr=subprocess.PIPE)
for line in validate_process.stdout.decode('utf-8').split('\n'):
    print(line)
for line in validate_process.stderr.decode('utf-8').split('\n'):
    print(line)